In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
## extra imports to set GPU options
import tensorflow as tf
from keras import backend as k

###################################
# TensorFlow wizardry
config = tf.ConfigProto()

# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 1 # 0.5 for Half

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))


Using TensorFlow backend.


In [3]:
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Activation
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from keras.utils import plot_model

import tensorflow as tf
import keras
import seaborn as sn
import pandas as pd
import numpy as np
import linecache
import sys
import h5py
import itertools
import matplotlib.pyplot as plt
import os
import pickle
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

In [4]:
tf.__version__

'1.9.0'

In [5]:
train_path = '/data/danish/Data/Malimg-Gray-25-Families/Malimg_Grayscale_train/'
test_path  = '/data/danish/Data/Malimg-Gray-25-Families/Malimg_Grayscale_test/'

In [6]:
datagen_train = ImageDataGenerator(
      rescale=1./255#,
      #rotation_range=0.1,#180,
      #width_shift_range=0.1,
      #height_shift_range=0.1,
      #shear_range=0.1,
      #zoom_range=0.1,#[0.9, 1.5],
      #horizontal_flip=True,
      #vertical_flip=True,
      #fill_mode='nearest'
                )

datagen_test = ImageDataGenerator(rescale=1./255)


In [7]:
if False:
    save_to_dir='augmented_images/'
else:
    save_to_dir = None

In [8]:
input_shape=(224,224)
batch_size=32
save_fn='Deep_Learning_Model_for_Malimg_size_GrayScale='+str(input_shape[:])
save_fn

'Deep_Learning_Model_for_Malimg_size_GrayScale=(224, 224)'

In [9]:
generator_train = datagen_train.flow_from_directory(directory=train_path,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    save_to_dir=save_to_dir,
                                                    seed=42)


steps_train = generator_train.n/ batch_size
print("Steps for Training",steps_train)

cls_train = generator_train.classes
print(cls_train)
cls_names_train = list(generator_train.class_indices.keys())
print(cls_names_train)
num_cls_train = len(cls_names_train)
print("Number of Classes in Training Data",num_cls_train)

Found 6527 images belonging to 25 classes.
Steps for Training 203.96875
[ 0  0  0 ..., 24 24 24]
['Adialer.C', 'Agent.FYI', 'Allaple.A', 'Allaple.L', 'Alueron.gen!J', 'Autorun.K', 'C2LOP.P', 'C2LOP.gen!g', 'Dialplatform.B', 'Dontovo.A', 'Fakerean', 'Instantaccess', 'Lolyda.AA1', 'Lolyda.AA2', 'Lolyda.AA3', 'Lolyda.AT', 'Malex.gen!J', 'ObfuscatorAD', 'Rbot!gen', 'Skintrim.N', 'Swizzorgen!E', 'Swizzorgen!I', 'VB.AT', 'Wintrim.BX', 'Yuner.A']
Number of Classes in Training Data 25


In [10]:
from sklearn.utils.class_weight import compute_class_weight
class_weight_train = compute_class_weight(class_weight='balanced',
                                    classes=np.unique(cls_train),
                                    y=cls_train)

In [11]:
#cls_names_train
class_labels = ['AdialerC','AgentFYI','AllapleA','AllapleL','Aluerongen!J','AutorunK','C2LOPP','C2LOPgen!g','DialplatformB','DontovoA',
 'Fakerean','Instantaccess','LolydaAA1','LolydaAA2','LolydaAA3','LolydaAT','Malexgen!J','ObfuscatorAD','Rbot!gen','SkintrimN',
 'Swizzorgen!E','Swizzorgen!I','VBAT','WintrimBX','YunerA']
class_labels

['AdialerC',
 'AgentFYI',
 'AllapleA',
 'AllapleL',
 'Aluerongen!J',
 'AutorunK',
 'C2LOPP',
 'C2LOPgen!g',
 'DialplatformB',
 'DontovoA',
 'Fakerean',
 'Instantaccess',
 'LolydaAA1',
 'LolydaAA2',
 'LolydaAA3',
 'LolydaAT',
 'Malexgen!J',
 'ObfuscatorAD',
 'Rbot!gen',
 'SkintrimN',
 'Swizzorgen!E',
 'Swizzorgen!I',
 'VBAT',
 'WintrimBX',
 'YunerA']

In [12]:
generator_test = datagen_test.flow_from_directory(directory=test_path,
                                                  target_size=input_shape,
                                                  class_mode='categorical',
                                                  batch_size=batch_size,
                                                  shuffle=False,
                                                  seed=42)

steps_test = generator_test.n / batch_size
steps_test
print("Steps for Test",steps_test)

cls_test = generator_test.classes
print(cls_test)
cls_names_test = list(generator_test.class_indices.keys())
print(cls_names_test)
num_cls_test = len(cls_names_test)
print("Number of Classes in Test Data",num_cls_test)

Found 2812 images belonging to 25 classes.
Steps for Test 87.875
[ 0  0  0 ..., 24 24 24]
['Adialer.C', 'Agent.FYI', 'Allaple.A', 'Allaple.L', 'Alueron.gen!J', 'Autorun.K', 'C2LOP.P', 'C2LOP.gen!g', 'Dialplatform.B', 'Dontovo.A', 'Fakerean', 'Instantaccess', 'Lolyda.AA1', 'Lolyda.AA2', 'Lolyda.AA3', 'Lolyda.AT', 'Malex.gen!J', 'ObfuscatorAD', 'Rbot!gen', 'Skintrim.N', 'Swizzorgen!E', 'Swizzorgen!I', 'VB.AT', 'Wintrim.BX', 'Yuner.A']
Number of Classes in Test Data 25


In [13]:
model_ = keras.applications.vgg16.VGG16()#include_top=True, weights=None, input_tensor=None, input_shape=(224,224,1), classes=num_cls_train)

In [14]:
transfer_layer = model_.get_layer('fc2')

In [15]:
transfer_layer.output

<tf.Tensor 'fc2/Relu:0' shape=(?, 4096) dtype=float32>

In [16]:
conv_model = Model(inputs=model_.input,
                   outputs=transfer_layer.output)

In [17]:
conv_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [18]:
for layer in conv_model.layers:
    
    
    # Boolean whether this layer is trainable.
    trainable = ('fc1' in layer.name or 'fc2' in layer.name or 'block1' in layer.name or 'block2' in layer.name or 'block3' in layer.name or 'block4' in layer.name or 'block5' in layer.name) # or 'block6' in layer.name or 'block7' in layer.name or 'block8' in layer.name or 'block9' in layer.name or 'block10' in layer.name or 'block11' in layer.name or 'block12' in layer.name or 'block13' in layer.name or 'block14' in layer.name)
    
    if trainable:
        layer.trainable = False #trainable
        #print("False")
        
    trainable_ = ('fc1' in layer.name or 'fc2' in layer.name or 'block5' in layer.name) #or 'block5' in layer.name or 'block6' in layer.name or 'block7' in layer.name or 'block8' in layer.name or 'block9' in layer.name or 'block10' in layer.name or 'block11' in layer.name or 'block12' in layer.name or 'block13' in layer.name or 'block14' in layer.name)
    
    if trainable_:
        # Set the layer's bool.
        layer.trainable = trainable_
        print(trainable_)

True
True
True
True
True
True


In [19]:
def print_layer_trainable():
    for layer in conv_model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))
print_layer_trainable()

False:	input_1
False:	block1_conv1
False:	block1_conv2
False:	block1_pool
False:	block2_conv1
False:	block2_conv2
False:	block2_pool
False:	block3_conv1
False:	block3_conv2
False:	block3_conv3
False:	block3_pool
False:	block4_conv1
False:	block4_conv2
False:	block4_conv3
False:	block4_pool
True:	block5_conv1
True:	block5_conv2
True:	block5_conv3
True:	block5_pool
True:	flatten
True:	fc1
True:	fc2


In [20]:
# Start a new Keras Sequential model.
model = Sequential()

# Add the convolutional part of the VGG16 model from above.
model.add(conv_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
#model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
#model.add(Dense(1024, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
model.add(Dropout(0.5))

# Add the final layer for the actual classification.
model.add(Dense(num_cls_train, activation='softmax'))

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 4096)              134260544 
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 25)                102425    
Total params: 134,362,969
Trainable params: 126,727,705
Non-trainable params: 7,635,264
_________________________________________________________________


In [22]:
from keras.models import load_model

#model = load_model('IMCFN-Grayscale.h5')
model.compile(Adam(lr=0.0001),loss='categorical_crossentropy', metrics=['accuracy'])


In [23]:
epochs =10
steps_per_epoch=steps_train

In [24]:


history = model.fit_generator(generator=generator_train,
                                epochs=epochs,
                                steps_per_epoch=steps_per_epoch,
                                validation_data=generator_test,
                                validation_steps=steps_test
                                )




Epoch 1/10
204/203 [==============================] - 58s 282ms/step - loss: 0.5140 - acc: 0.8684 - val_loss: 0.1531 - val_acc: 0.9502
Epoch 2/10
204/203 [==============================] - 45s 220ms/step - loss: 0.1115 - acc: 0.9632 - val_loss: 0.1060 - val_acc: 0.9683
Epoch 3/10
204/203 [==============================] - 45s 221ms/step - loss: 0.0880 - acc: 0.9712 - val_loss: 0.0942 - val_acc: 0.9701
Epoch 4/10
204/203 [==============================] - 45s 220ms/step - loss: 0.0616 - acc: 0.9798 - val_loss: 0.1368 - val_acc: 0.9552
Epoch 5/10
204/203 [==============================] - 45s 221ms/step - loss: 0.0774 - acc: 0.9802 - val_loss: 0.0704 - val_acc: 0.9762
Epoch 6/10
204/203 [==============================] - 45s 221ms/step - loss: 0.0522 - acc: 0.9867 - val_loss: 0.0657 - val_acc: 0.9819
Epoch 7/10
204/203 [==============================] - 44s 218ms/step - loss: 0.0339 - acc: 0.9891 - val_loss: 0.0918 - val_acc: 0.9758
Epoch 8/10
204/203 [==============================] - 4

In [25]:
model.save("No-Aug-Grayscale.h5")

In [26]:
#####################################
# write Traning History to a file
output = open('No-Aug-Grayscale-History.pkl', 'wb')
pickle.dump(history.history, output)
output.close()
#####################################

In [27]:

# read Traning History back from the file
pkl_file = open('No-Aug-Grayscale-History.pkl', 'rb')
history = pickle.load(pkl_file)
pkl_file.close()

In [28]:
import pandas as pd
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history) 

csv_file = 'No-Aug-Grayscale.csv'
with open(csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [29]:
#scores = model.evaluate(X[test], Y[test], verbose=0)
#model.load_weights('Model_VGG16_Malimg_RGB_Trained_(229, 229)_Epochs=100_Model_Traning_Testing_Weights.hdf5')
scores = model.evaluate_generator(generator_test, verbose=1)# steps_test, max_queue_size=10, workers=1, use_multiprocessing=False,verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

88/88 [==============================] - 11s 129ms/step
acc: 98.04%


In [30]:
cls_pred = model.predict_generator(generator_test, verbose=1)#, steps_test, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)

88/88 [==============================] - 11s 130ms/step


In [31]:
cls_pred = np.argmax(cls_pred, axis=1)

In [32]:
y_test = cls_test = generator_test.classes

from pycm import ConfusionMatrix
CM_IMCFN = ConfusionMatrix(y_test,cls_pred)
file = open('CM-No-Aug-Grayscale.pkl', 'wb')
pickle.dump(CM_IMCFN, file)
file.close()

In [33]:
# write Traning History to a file
file = open('CM-No-Aug-Grayscale.pkl', 'rb')
CM_IMCFN = pickle.load(file)
file.close()

In [34]:
CM_IMCFN.stat(summary=True)

Overall Statistics : 

ACC Macro                                                         0.99844
F1 Macro                                                          0.95258
Kappa                                                             0.97709
Overall ACC                                                       0.98044
PPV Macro                                                         0.95464
SOA1(Landis & Koch)                                               Almost Perfect
TPR Macro                                                         0.95211
Zero-one Loss                                                     55

Class Statistics :

Classes                                                           0             1             10            11            12            13            14            15            16            17            18            19            2             20            21            22            23            24            3             4             5             6 

/home/danish/anaconda3/envs/tf/lib/python3.6/site-packages/pycm/pycm_obj.py:167: RuntimeWarning: The confusion matrix is a high dimension matrix and won't be demonstrated properly.
The save_csv method can be used to save the confusion matrix in csv format and have a better demonstration of it.
  warn(CLASS_NUMBER_WARNING, RuntimeWarning)


In [35]:
CM_IMCFN.save_csv("CM-No-Aug-Grayscale",address=True)

{'Status': True,
 'Message': '/data/danish/ImgProPython/TensorFlow-Tutorials/Computer-Networks-(CCF-B)/No-Aug-Grayscale/CM-No-Aug-Grayscale.csv'}

In [36]:
%%time
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
cls_true = cls_test = generator_test.classes
accuracy = accuracy_score(cls_true, cls_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(cls_true, cls_pred, average=None)

print("All Classes")
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

All Classes
Precision:  [ 1.          1.          0.99774521  1.          1.          1.
  0.86363636  0.95833333  1.          1.          1.          1.
  0.98412698  0.96491228  1.          1.          1.          1.          1.
  1.          0.54166667  0.55555556  1.          1.          1.        ]
Recall:  [ 1.          1.          1.          1.          1.          1.
  0.86363636  0.76666667  0.96296296  1.          0.99130435  1.          0.96875
  0.98214286  1.          1.          0.97560976  1.          1.          1.
  0.66666667  0.625       1.          1.          1.        ]
F1 score:  [ 1.          1.          0.99887133  1.          1.          1.
  0.86363636  0.85185185  0.98113208  1.          0.99563319  1.
  0.97637795  0.97345133  1.          1.          0.98765432  1.          1.
  1.          0.59770115  0.58823529  1.          1.          1.        ]
CPU times: user 14.9 ms, sys: 3.87 ms, total: 18.7 ms
Wall time: 15.3 ms


In [37]:
%%time
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
cls_true = cls_test = generator_test.classes
accuracy = accuracy_score(cls_true, cls_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(cls_true, cls_pred, average='micro')

print("Micro")
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

Micro
Precision:  0.980440967283
Recall:  0.980440967283
F1 score:  0.980440967283
CPU times: user 12.1 ms, sys: 542 µs, total: 12.6 ms
Wall time: 24 ms


In [38]:
%%time
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
cls_true = cls_test = generator_test.classes
accuracy = accuracy_score(cls_true, cls_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(cls_true, cls_pred, average='macro')

print("Macro")
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

Macro
Precision:  0.954639055704
Recall:  0.95210958484
F1 score:  0.952581794211
CPU times: user 4.89 ms, sys: 3.61 ms, total: 8.5 ms
Wall time: 8.44 ms


In [39]:
%%time
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
cls_true = cls_test = generator_test.classes
accuracy = accuracy_score(cls_true, cls_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(cls_true, cls_pred, average='weighted')

print("weighted")
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

weighted
Precision:  0.982528786098
Recall:  0.980440967283
F1 score:  0.9811259728
CPU times: user 5.72 ms, sys: 0 ns, total: 5.72 ms
Wall time: 6.95 ms
